# Amazon Bedrock Fundamental

> 이 노트북은, SageMaker Notebook <i><b>conda_python3</b></i> 커널에서 테스트 되었습니다.

## 1. 환경 설정

In [11]:
# 처음 실행하는 경우, 주석(#)을 제거하고 필요한 라이브러리들을 이 환경에 설치합니다.

#!pip install -r ./requirements.txt

## 2. Bedrock Client 생성

In [14]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" # Claude 3 Sonnet ModelId
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock')
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)  # 사용 가능한 Bedrock의 모델 리스트

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.clau

## 3. Claude 3 모델 로딩

In [15]:
def image_to_base64(img):
    import base64
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")


def bedrock_streamer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
    return answer

In [17]:
from botocore.config import Config

session = boto3.Session()

retry_config = Config(
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)


bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=retry_config
)

In [18]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
image_path = None  # 프롬프트에 함께 입력할 이미지가 있는 경우, 이미지 path 입력

In [19]:
if image_path is None:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }
else:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_to_base64(image_path),
                        },
                    },
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }


body = json.dumps(prompt_config)

In [21]:
from IPython.display import clear_output, display, display_markdown, Markdown

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    results=bedrock_streamer(response)

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 법적 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며 피보험자의 생존기간 동안 보장됩니다.
- 손해보험은 단기계약이 일반적이며 1년 단위로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가족력 등 인적위험을 기준으로 산정됩니다.
- 손해보험료는 보험대상의 가액, 위험률, 손해율 등 물적위험을 기준으로 산정됩니다.

요약하면 생명보험은 개인의 생명과 관련된 위험을 장기적으로 보장하고, 손해보험은 재산상의 손해나 배상책임을 단기적으로 보상하는 차이가 있습니다.

---

## 4. LangChain을 활용한 Bedrock Claude 3 사용 예시


#### 4-1. LangChain의 BedrockChat을 활용한 가장 간단한 코드 예시

In [23]:
import os
from langchain_community.chat_models import BedrockChat

def get_text_response(input_content):
    llm = BedrockChat(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"),
        region_name=os.environ.get("BWB_REGION_NAME"),
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs={
            "max_tokens": 512,
            "temperature": 0,
            "top_p": 0.01,
            "top_k": 0,
        }
    )
    return llm.predict(input_content)

if __name__ == "__main__":
    input_text = input("Input text: ")
    response_content = get_text_response(input_content=input_text)
    print(response_content)

Input text:  당신은 누구 입니까?


안녕하세요. 저는 Claude라는 인공지능 대화 모델입니다. 제 역할은 사람들과 대화하며 궁금한 점을 해결하고 도움을 주는 것입니다. 인공지능이지만 인간처럼 생각하고 대화할 수 있도록 훈련되었습니다. 어떤 분야에 대해 궁금하신 점이 있으시면 물어봐 주세요.


#### 4-2. LangChain의 BedrockChat을 활용한 챗봇 기능(이어지는 대화 가능)을 구현한 코드 예시

In [ ]:
import os
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import BedrockChat
from langchain.chains import ConversationChain

def get_llm():
    model_kwargs = {
        "max_tokens": 1024,
        "temperature": 0,
        "top_p": 0.5,
        "top_k": 0,
        "stop_sequences": ["\n\nHuman:"],
        "messages": [{"role": "system", "content": "You are a helpful assistant."}]
    }

    llm = BedrockChat(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"),
        region_name=os.environ.get("BWB_REGION_NAME"),
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs=model_kwargs)

    return llm

def get_memory():
    llm = get_llm()
    chat_memory = ConversationBufferWindowMemory(human_prefix='Human', ai_prefix='Assistant')
    conversation = ConversationChain(llm=llm, verbose=False, memory=chat_memory)
    return chat_memory

def get_chat_response(input_text, memory):
    llm = get_llm()
    conversation_with_summary = ConversationChain(llm=llm, memory=memory, verbose=True)
    chat_response = conversation_with_summary.predict(input=input_text)
    return chat_response

if __name__ == "__main__":
    memory = get_memory()
    chat_history = []

    while True:
        input_text = input("Human: ")
        print("Human:", input_text)
        chat_history.append({"role": "user", "text": input_text})

        chat_response = get_chat_response(input_text=input_text, memory=memory)
        print("Assistant:", chat_response)
        chat_history.append({"role": "assistant", "text": chat_response})

Human:  안녕하세요.


Human: 안녕하세요.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요.
AI:

> Finished chain.
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?


Human:  당신의 이름은 무엇인가요?


Human: 당신의 이름은 무엇인가요?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요.
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?
Human: 당신의 이름은 무엇인가요?
AI:

> Finished chain.
Assistant: 안녕하세요. 제 이름은 Claude라고 말씀드렸습니다. 프랑스 과학자 Claude Shannon의 이름에서 따온 것 같아요. 그분은 정보이론과 디지털 회로 분야에서 큰 업적을 남기셨죠. 하지만 정확한 이유는 모르겠네요. 궁금하시다면 제작사에 문의해보시는 것도 좋을 것 같아요.


Human:  무지개의 세번째 색은 무엇인가요?


Human: 무지개의 세번째 색은 무엇인가요?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요.
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?
Human: 당신의 이름은 무엇인가요?
Assistant: 안녕하세요. 제 이름은 Claude라고 말씀드렸습니다. 프랑스 과학자 Claude Shannon의 이름에서 따온 것 같아요. 그분은 정보이론과 디지털 회로 분야에서 큰 업적을 남기셨죠. 하지만 정확한 이유는 모르겠네요. 궁금하시다면 제작사에 문의해보시는 것도 좋을 것 같아요.
Human: 무지개의 세번째 색은 무엇인가요?
AI:

> Finished chain.
Assistant: 무지개의 7가지 색깔 순서는 빨강, 주황, 노랑, 초록, 파랑, 남색, 보라색입니다. 그러므로 무지개의 세 번째 색은 노랑색이 됩니다.


Human:  다음 색은 뭐에요?


Human: 다음 색은 뭐에요?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕하세요.
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?
Human: 당신의 이름은 무엇인가요?
Assistant: 안녕하세요. 제 이름은 Claude라고 말씀드렸습니다. 프랑스 과학자 Claude Shannon의 이름에서 따온 것 같아요. 그분은 정보이론과 디지털 회로 분야에서 큰 업적을 남기셨죠. 하지만 정확한 이유는 모르겠네요. 궁금하시다면 제작사에 문의해보시는 것도 좋을 것 같아요.
Human: 무지개의 세번째 색은 무엇인가요?
Assistant: 무지개의 7가지 색깔 순서는 빨강, 주황, 노랑, 초록, 파랑, 남색, 보라색입니다. 그러므로 무지개의 세 번째 색은 노랑색이 됩니다.
Human: 다음 색은 뭐에요?
AI:

> Finished chain.
Assistant: 무지개 색깔 순서대로 노랑 다음에 오는 색은 초록색입니다.
